## SigNet Package test

**General idea** : test the functions defined in the module1.py file and make a first application.

**General plan** : 
1. Import the data using the DropBox dataset

2. Repeat the clustering several times to get several clustering outcomes

3. Compute the weights of each stocks within each cluster 

4. Compute the returns of each portfolio, where one portfolio corresponds to one clustering and is composed of 5 big assets (which are the 5 clusters) **in this notebook, we first fix the number of clusters to be equal to 5**

Here are the main package we use in this notebook

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from bs4 import BeautifulSoup
import requests 
from pypfopt.efficient_frontier import EfficientFrontier

We also import the module1.py file to use the function we defined there.

In [2]:
## we make some manipulations to correctly import module1

import os
import sys

# on a obtenu le chemin absolu vers notre répertoire en utilisant
# le terminal (ls, pwd, cd)
module_1_directory = '/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/code'

# Ajouter le chemin du répertoire 'code' au chemin de recherche Python
sys.path.append(module_1_directory)

# Maintenant, vous pouvez importer module1
import module1

We also import the adjency.py file to use the function we defined there.

In [3]:
## we make some manipulations to correctly import module1

import os
import sys

# on a obtenu le chemin absolu vers notre répertoire en utilisant
# le terminal (ls, pwd, cd)
adjency_directory = '/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/code'

# Ajouter le chemin du répertoire 'code' au chemin de recherche Python
sys.path.append(adjency_directory)

# Maintenant, vous pouvez importer module1
from adjency import Adjency

### 1. Importation of data

We download various types of data (Open, High, Low, Close, Adj Close, Volume) for **496** assets between two periods (start, end) into a pandas dataframe denoted as df.

We then store the returns of each of these assets in the 'data'.

In [41]:
## beware of the path if Jerome or Mohamed use it 

df = pd.read_csv('/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DATA_Statapp.csv')

In [46]:
## on met les tickers en nom d'index 

data = df[['ticker', 'open', 'close', 'volume']]
data.set_index('ticker', inplace=True)

In [66]:
## L'objectif est ici de créer une colonne returns à partir d'opérations sur les colonnes open et close. La difficulté vient du fait que chaque élément est
## un string composé d'une liste (on est obligé de passer par là pour pouvoir stocker une liste par élément du dataframe)

number_stocks = len(data.index) ## on récupère le nombre de stocks 
data['return'] = np.nan ## on instancie la colonne return avec des NaN avant de la remplir au fil de la boucle 
for i in range(number_stocks):

    ## quelques manipulations sont requises pour se débarrasser des strings 
    open = np.array(data.iloc[i][0].replace('[', '').replace(']', '').split(', '), dtype=float)
    close = np.array(data.iloc[i][1].replace('[', '').replace(']', '').split(', '), dtype=float)
    returns = (close - open) / open

    data.iloc[i, 3] = str(returns.tolist()) ## la troisième colonne correspond à la colonne returns 

/var/folders/q2/wg5gyfhj2r9cd97zfmckktvw0000gn/T/ipykernel_8183/1516761896.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['return'] = np.nan ## on instancie la colonne return avec des NaN avant de la remplir au fil de la boucle
/var/folders/q2/wg5gyfhj2r9cd97zfmckktvw0000gn/T/ipykernel_8183/1516761896.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  open = np.array(data.iloc[i][0].replace('[', '').replace(']', '').split(', '), dtype=float)
/var/folders/q2/wg5gyfhj2r9cd97zfmckktvw0000gn/T/ipykernel_8183/1516761896.py:10: FutureWarning: Series.__getit

In [83]:
def sub_data_slicing(data, m, k):
    '''
    L'objectif de cette fonction est de créer un sous dataframe du dataframe data qui ne garde que m jours de returns pour chaque stock 

    PARAMS : 

    - data : donnéees
    - m : nombre de jours considérés dans le slicing
    - k : numéro du slicing (k+1-ème fois que l'on décale) !! on commence à 0 le slicing
    '''


    sub_data_k = pd.DataFrame(index=data.index, columns=data.columns)

    number_stocks = len(data.index)

    for i in range(number_stocks):

        ## on récupère à chaque fois les 4 colonnes du dataframe, on slice pour récupérer m jours selon le k-ème slicing (PAS DE CHEVAUCHEMENT), on 
        ## transforme le tout en liste puis en string
        sub_data_k.iloc[i, 0] = str(np.array(data.iloc[i][0].replace('[', '').replace(']', '').split(', '), dtype=float)[k*m:(k+1)*m].tolist())
        sub_data_k.iloc[i, 1] = str(np.array(data.iloc[i][1].replace('[', '').replace(']', '').split(', '), dtype=float)[k*m:(k+1)*m].tolist())
        sub_data_k.iloc[i, 2] = str(np.array(data.iloc[i][2].replace('[', '').replace(']', '').split(', '), dtype=float)[k*m:(k+1)*m].tolist())
        sub_data_k.iloc[i, 3] = str(np.array(data.iloc[i][3].replace('[', '').replace(']', '').split(', '), dtype=float)[k*m:(k+1)*m].tolist())

    return sub_data_k

In [84]:
## on considère m = 250 jours 

sub_data_slicing_1 = sub_data_slicing(data=data, m=250, k=0) ## attention le slicing commence à 0
sub_data_slicing_2 = sub_data_slicing(data=data, m=250, k=1)

/var/folders/q2/wg5gyfhj2r9cd97zfmckktvw0000gn/T/ipykernel_8183/693824921.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sub_data_k.iloc[i, 0] = str(np.array(data.iloc[i][0].replace('[', '').replace(']', '').split(', '), dtype=float)[k*m:(k+1)*m].tolist())
/var/folders/q2/wg5gyfhj2r9cd97zfmckktvw0000gn/T/ipykernel_8183/693824921.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sub_data_k.iloc[i, 1] = str(np.array(data.iloc[i][1].replace('[', '').replace(']', '').split(', '), dtype=float)[k*m:(k+1)*m].tolist())
/var/folders/q2/wg5gyfhj2r9cd97zfmckktvw0000gn/T/ipykernel_8183/693824921.py:23: FutureWarning: Se

Désormais que tout est prêt au niveau du slicing et du nettoyage des données, il nous reste à pouvoir calculer les matrices d'adjacence associées à chaque sub_data_slicing puis à implémenter le clustering dans le package SigNet

#### Idea - Similarity

A popular similarity measure in the literature is given by the **Pearson correlation coefficient** that measures linear dependence between variables and takes values in [−1, 1]. By interpreting the correlation matrix as a weighted network whose (signed) edge weights capture the pairwise correlations, we cluster the multivariate time series by clustering the underlying signed network. 

Pearson's correlation coefficient, when applied to a [[sample (statistics)|sample]], is commonly represented by $r_{xy}$ and may be referred to as the ''sample correlation coefficient'' or the ''sample Pearson correlation coefficient''. We can obtain a formula for $r_{xy}$ by substituting estimates of the covariances and variances based on a sample into the formula above. Given paired data $\left\{ (x_1,y_1),\ldots,(x_n,y_n) \right\}$ consisting of $n$ pairs, $r_{xy}$ is defined as

\begin{align}
r_{xy} =\frac{\sum ^n _{i=1}(x_i - \bar{x})(y_i - \bar{y})}{\sqrt{\sum ^n _{i=1}(x_i - \bar{x})^2} \sqrt{\sum ^n _{i=1}(y_i - \bar{y})^2}}
\end{align}

where
- $n$ is sample size
- $x_i, y_i$ are the individual sample points indexed with $i$
- $\bar{x} = \frac{1}{n} \sum_{i=1}^n x_i$ (the sample mean); and analogously for $\bar{y}$.

In [89]:
def pearson_correlation(data):


    number_stocks = len(data.index)

    A = np.zeros((number_stocks, number_stocks))

    for i in range(number_stocks):

        for j in range(i, number_stocks): ## car la matrice A sera symétrique

            if i == j:
                A[i, j] = 1
            
            else:
                ## attention, le code n'est pas très générique ici car on suppose que c'est la troisième colonne 
                ## du dataframe data entré en argument qui contient les données sur les returns
                returns_1 = np.array(data.iloc[i,3].replace('[', '').replace(']', '').split(', '), dtype=float)
                returns_2 = np.array(data.iloc[j, 0].replace('[', '').replace(']', '').split(', '), dtype=float)
                A[i, j] = np.corrcoef(returns_1, returns_2)[0, 1]
    
    return pd.DataFrame(A + A.transpose() - np.eye(number_stocks))

In [90]:
correlation_matrix_1 = pearson_correlation(sub_data_slicing_1)

In [93]:
correlation_matrix_1

,0,1,2,3,4,5,6,7,8,9,...,654,655,656,657,658,659,660,661,662,663
0,1.000000,0.124747,0.096005,-0.104243,0.024935,-0.120584,0.060576,-0.071991,-0.046851,0.081453,...,0.026084,-0.097194,0.091137,0.103709,-0.075844,-0.095874,0.008330,-0.116348,-0.010924,0.016789
1,0.124747,1.000000,0.132588,-0.044521,0.046348,-0.117968,0.126917,-0.126336,-0.064732,0.141934,...,0.050512,-0.128076,0.125788,0.122749,-0.098049,-0.093056,0.071754,-0.141819,-0.005190,-0.015522
2,0.096005,0.132588,1.000000,-0.083501,-0.123678,0.000747,-0.031094,0.042462,-0.099844,-0.030388,...,0.008553,-0.009336,-0.043670,0.004492,-0.055257,0.005406,0.056858,0.028547,-0.042840,0.074267
3,-0.104243,-0.044521,-0.083501,1.000000,0.039352,0.004479,0.041708,-0.004358,0.101048,0.048480,...,-0.060096,-0.057270,0.013832,-0.033977,-0.062804,-0.011303,0.002942,-0.016627,-0.022719,-0.081809
4,0.024935,0.046348,-0.123678,0.039352,1.000000,-0.193873,0.099445,0.056878,-0.125763,0.139407,...,0.046078,-0.190962,0.153213,0.157185,-0.133564,-0.009165,0.159740,-0.079543,0.172067,0.004246
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
659,-0.095874,-0.093056,0.005406,-0.011303,-0.009165,0.034194,0.010300,-0.029239,0.000802,-0.028263,...,-0.077848,-0.014481,-0.110007,-0.083268,-0.095057,1.000000,0.002685,-0.048460,-0.083224,0.017714
660,0.008330,0.071754,0.056858,0.002942,0.159740,-0.076665,-0.010315,-0.073714,0.046390,0.086273,...,0.049480,-0.069666,0.092894,0.026978,0.034323,0.002685,1.000000,0.043339,-0.012151,-0.081628
661,-0.116348,-0.141819,0.028547,-0.016627,-0.079543,0.086572,-0.018668,0.037909,-0.020903,-0.078098,...,-0.034196,0.050122,-0.069385,-0.002717,-0.043909,-0.048460,0.043339,1.000000,-0.015491,-0.055018
662,-0.010924,-0.005190,-0.042840,-0.022719,0.172067,-0.021062,0.066470,-0.139308,0.115735,0.060671,...,-0.045718,-0.150913,-0.051309,-0.058920,-0.072849,-0.083224,-0.012151,-0.015491,1.000000,0.045507


In [92]:
def signed_adjency(mat):
    '''
    L'idée est ici, à partir d'une matrice de corrélation mat, de renvoyer deux matrices 
    A_positive et A_negative qui correspondraient aux matrices des corrélations positives et négatives 
    associées  
    '''

    A_pos = mat.applymap(lambda x: x if x >= 0 else 0)
    A_neg = mat.applymap(lambda x: abs(x) if x < 0 else 0)
    
    return A_pos, A_neg


In [94]:
A_pos, A_neg = signed_adjency(correlation_matrix_1)

/var/folders/q2/wg5gyfhj2r9cd97zfmckktvw0000gn/T/ipykernel_8183/2426464110.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  A_pos = mat.applymap(lambda x: x if x >= 0 else 0)
/var/folders/q2/wg5gyfhj2r9cd97zfmckktvw0000gn/T/ipykernel_8183/2426464110.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  A_neg = mat.applymap(lambda x: abs(x) if x < 0 else 0)


#### **Mise en application avec SigNet**

On utilise désormais le package signet dont une documentation peut-être trouvée à l'adresse suivante : 

https://github.com/alan-turing-institute/SigNet/blob/master/README.md

On a copié tout le code de signet dans le fichier signet et on importe désormais tout le code dans ce fichier

In [100]:
import sys

sys.path.append('/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project')  # Ajoute le chemin parent

from signet.cluster import Cluster 

In [116]:
from scipy import sparse

def apply_SPONGE(correlation_matrix, k): 

    '''
    IDÉE : étant donné une matrice de correlation obtenue à partir d'une base de donnée et de la similarité de pearson, renvoyer un vecteur associant 
           à chaque actif le numéro du cluster auquel il appartient une fois qu'on lui a appliqué SPONGE (à partir du package signet)

    PARAMS : 

    - correlation_matrix : a square dataframe of size (number_of_stocks, number_of_stocks)
    - k : the number of clusters to identify. If a list is given, the output is a corresponding list

    RETURNS : array of int, or list of array of int: Output assignment to clusters.

    '''
    
    ## On respecte le format imposé par signet. Pour cela il faut changer le type des matrices A_pos et A_neg, qui ne peuvent pas rester des dataframes 

    A_pos, A_neg = signed_adjency(correlation_matrix)

    A_pos_sparse = sparse.csc_matrix(A_pos.values)
    A_neg_sparse = sparse.csc_matrix(A_neg.values)

    data = (A_pos_sparse, A_neg_sparse)

    cluster = Cluster(data)

    ## On applique la méthode SPONGE : clusters the graph using the Signed Positive Over Negative Generalised Eigenproblem (SPONGE) clustering.

    return cluster.SPONGE(k = 20)


In [117]:
result = apply_SPONGE(correlation_matrix_1, k=20)

/var/folders/q2/wg5gyfhj2r9cd97zfmckktvw0000gn/T/ipykernel_8183/2426464110.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  A_pos = mat.applymap(lambda x: x if x >= 0 else 0)
/var/folders/q2/wg5gyfhj2r9cd97zfmckktvw0000gn/T/ipykernel_8183/2426464110.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  A_neg = mat.applymap(lambda x: abs(x) if x < 0 else 0)
/usr/local/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [118]:
result

array([10, 10,  9, 12, 17,  9, 10, 17, 17, 19, 13, 10, 11,  9, 11,  9, 18,
        9,  9, 15, 10, 11, 10,  9,  9, 12, 12, 18,  9, 10,  9,  9, 11, 19,
       12, 12, 17, 14,  3,  9, 15, 18, 18, 11, 19, 19, 19, 11, 17, 15, 15,
       15, 10, 10, 15, 16,  9, 10, 12, 18, 11, 12, 16, 10, 15, 11, 11,  6,
        9,  9, 12, 11, 18, 14,  9, 15, 19, 17, 12, 11, 15, 10, 10, 11, 10,
       10, 19,  9,  9, 11, 14, 18, 10, 11, 12,  9,  0, 10, 11, 11, 11,  0,
       11,  9, 12, 11, 10, 11, 10, 18, 10, 18,  9,  0,  9, 19, 11,  9, 15,
       19, 18, 17,  0, 17,  9, 11,  0, 15, 17, 18, 17, 15, 11, 10, 15, 15,
       12, 18,  9, 11, 10, 11,  9,  9, 15,  1,  0, 12, 15, 10,  9,  9, 15,
       14, 15,  9, 11, 15, 12,  0, 10, 15, 17, 15, 19,  4, 12, 19,  0, 10,
       18, 11, 11, 11, 15, 11, 11, 15,  0, 17, 15, 12, 10, 10, 18, 12, 12,
       18, 17, 17, 11, 19, 12, 10, 10, 18, 15, 10, 12,  9, 15, 12,  9, 11,
       15, 11, 11, 11, 10, 18, 18, 15, 18, 12, 11, 14, 17, 12,  9, 11, 10,
       15, 10, 11, 10, 11